In [ ]:
#install tools untuk tahap pre-processing
!pip install nltk
!pip install Sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import swifter
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# ! pip install Sastrawi
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# stop_factory = StopWordRemoverFactory()
# data = stop_factory.get_stop_words()
# stopword = stop_factory.create_stop_word_remover()
# # newStopWords = ['stopWord1','stopWord2']
# # stopwords.extend(newStopWords)
# print(data)

In [ ]:
# df = pd.DataFrame(data) 
    
# # saving the dataframe 
# df.to_csv('stopword_sastrawi.csv') 

In [ ]:
data = pd.read_csv("Data Skripsi Anne NoTgl.csv" , encoding='latin-1')

In [ ]:
data.head()

,Judul,Isi Berita,Label
0,Kapolri Minta Vaksinasi Booster untuk Lansia D...,CIANJUR - Kapolri Jenderal Listyo Sigit Prabow...,0
1,Sudah Vaksin dan Penyintas Covid-19 Masih Terp...,JAKARTA- Gejala pada infeksi Covid-19 variant ...,1
2,Infeksi Covid-19 Tak Hanya Berbahaya bagi Komo...,DOKTER Spesialis Penyakit Dalam Konsultan Peny...,0
3,MNC Peduli dan Lotte Mart Gelar Sentra Vaksin ...,JAKARTA - Lotte Mart bekerja sama dengan MNC P...,0
4,Kapolda Metro Tinjau Vaksinasi Lansia di Alun-...,"BEKASI - Kapolda Metro Jaya, Irjen Fadil Imran...",0


In [ ]:
data.isnull().sum().sum() #finding missing values

0

In [ ]:
#Case Folding
data['Judul'] = data['Judul'].str.lower()
data['Isi Berita'] = data['Isi Berita'].str.lower()
data.head()

,Judul,Isi Berita,Label
0,kapolri minta vaksinasi booster untuk lansia d...,cianjur - kapolri jenderal listyo sigit prabow...,0
1,sudah vaksin dan penyintas covid-19 masih terp...,jakarta- gejala pada infeksi covid-19 variant ...,1
2,infeksi covid-19 tak hanya berbahaya bagi komo...,dokter spesialis penyakit dalam konsultan peny...,0
3,mnc peduli dan lotte mart gelar sentra vaksin ...,jakarta - lotte mart bekerja sama dengan mnc p...,0
4,kapolda metro tinjau vaksinasi lansia di alun-...,"bekasi - kapolda metro jaya, irjen fadil imran...",0


In [ ]:
#Cleaning 
def cleaning_text(text):
    # Menghilangkan baris baru, tab, underline.
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # Ambil 3 Kalimat
    text = ' '.join(re.split(r'(?<=[.:;])\s', text)[:3])
    # Menghilangkan simbol emoticon dan huruf selain ascii
    text = text.encode('ascii', 'replace').decode('ascii')
    # Menghapus mention (@), link, dan hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove_empty_space
    text = re.sub(r'', '', text.strip())
    # remove number
    text = re.sub("\d+", "", text)
    #remove multiple space
    text = re.sub(r'\s\s+', ' ', text)
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

In [ ]:
data['Judul'] = data['Judul'].apply(lambda x: cleaning_text(x))
data['Isi Berita'] = data['Isi Berita'].apply(lambda x: cleaning_text(x))
data.head()

,Judul,Isi Berita,Label
0,kapolri minta vaksinasi booster untuk lansia d...,cianjur kapolri jenderal listyo sigit prabowo ...,0
1,sudah vaksin dan penyintas covid masih terpapa...,jakarta gejala pada infeksi covid variant omic...,1
2,infeksi covid tak hanya berbahaya bagi komorbi...,dokter spesialis penyakit dalam konsultan peny...,0
3,mnc peduli dan lotte mart gelar sentra vaksin ...,jakarta lotte mart bekerja sama dengan mnc ped...,0
4,kapolda metro tinjau vaksinasi lansia di aluna...,bekasi kapolda metro jaya irjen fadil imran di...,0


In [ ]:
#Tokenizing
def tokenization(text):
	text = re.split('\W+', text)
	return text

In [ ]:
data['Judul'] = data['Judul'].apply(lambda x: tokenization(x.lower()))
data['Isi Berita'] = data['Isi Berita'].apply(lambda x: tokenization(x.lower()))
data.head(10)

,Judul,Isi Berita,Label
0,"[kapolri, minta, vaksinasi, booster, untuk, la...","[cianjur, kapolri, jenderal, listyo, sigit, pr...",0
1,"[sudah, vaksin, dan, penyintas, covid, masih, ...","[jakarta, gejala, pada, infeksi, covid, varian...",1
2,"[infeksi, covid, tak, hanya, berbahaya, bagi, ...","[dokter, spesialis, penyakit, dalam, konsultan...",0
3,"[mnc, peduli, dan, lotte, mart, gelar, sentra,...","[jakarta, lotte, mart, bekerja, sama, dengan, ...",0
4,"[kapolda, metro, tinjau, vaksinasi, lansia, di...","[bekasi, kapolda, metro, jaya, irjen, fadil, i...",0
5,"[cirebon, ppkm, level, tapi, tingkat, hunian, ...","[cirebon, berdasarkan, asesmen, kementerian, k...",0
6,"[melawan, covid, psikolog, berpikir, sehat, da...","[psikolog, dr, novrans, eka, saputra, mengatak...",0
7,"[pandemi, covid, bnpb, fokus, pada, perubahan,...","[jakarta, kepala, badan, nasional, penanggulan...",0
8,"[kunci, sembuh, dari, covid, berpikir, positif...","[sejatinya, virus, corona, akan, hilang, denga...",1
9,"[indonesia, sudah, lewati, gelombang, ketiga, ...","[selama, satu, pekan, terakhir, terlihat, ada,...",1


In [ ]:
#Stop Removal

stopword = nltk.corpus.stopwords.words('indonesian')

stopword = [e for e in stopword if e not in ('akankah', 'apa', 'apakah', 'bagaimana', 'bagaimanakah', 'beberapa', 'begini', 'beginilah', 'benarkah', 'berapa', 'berapakah', 'berikut', 'bisakah', 'bolehkah', 'cara', 'hal', 'ini', 'inikah', 'inilah', 'kapan', 'kapankah', 'kok', 'mengapa', 'menurut', 'mungkinkah', 'pentingnya', 'siapa', 'siapakah', 'waduh', 'wah')]

print(stopword)

df = pd.DataFrame(stopword) 
    
# saving the dataframe 
# data.to_csv('stopword_sastrawi.csv') 

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apaan', 'apabila', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beginian', 'beginikah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan', 'berkenaan', 'berlainan', 'berlalu', 'berlangsung', 'berlebihan', 'bermacam', 'bermacam-macam', 'bermaksud', 'bermula', 'bersama', 'bers

In [ ]:
def remove_stopwords(text):
	text = [word for word in text if word not in stopword]
	return text

data['Judul'] = data['Judul'].apply(lambda x: remove_stopwords(x))
data['Isi Berita'] = data['Isi Berita'].apply(lambda x: remove_stopwords(x))
data.head(10)

,Judul,Isi Berita,Label
0,"[kapolri, vaksinasi, booster, lansia, dimaksim...","[cianjur, kapolri, jenderal, listyo, sigit, pr...",0
1,"[vaksin, penyintas, covid, terpapar, omicron]","[jakarta, gejala, infeksi, covid, variant, omi...",1
2,"[infeksi, covid, berbahaya, komorbid]","[dokter, spesialis, penyakit, konsultan, penya...",0
3,"[mnc, peduli, lotte, mart, gelar, sentra, vaks...","[jakarta, lotte, mart, mnc, peduli, menghelat,...",0
4,"[kapolda, metro, tinjau, vaksinasi, lansia, al...","[bekasi, kapolda, metro, jaya, irjen, fadil, i...",0
5,"[cirebon, ppkm, level, tingkat, hunian, rumah,...","[cirebon, berdasarkan, asesmen, kementerian, k...",0
6,"[melawan, covid, psikolog, berpikir, sehat, ce...","[psikolog, dr, novrans, eka, saputra, berpikir...",0
7,"[pandemi, covid, bnpb, fokus, perubahan, peril...","[jakarta, kepala, badan, nasional, penanggulan...",0
8,"[kunci, sembuh, covid, berpikir, positif, maka...","[sejatinya, virus, corona, hilang, tubuh, keke...",1
9,"[indonesia, lewati, gelombang, ketiga, covid, ...","[pekan, tren, penurunan, angka, positif, covid...",1


In [ ]:
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# # stemmed
# def stemmed_wrapper(term):
#     return stemmer.stem(term)

# term_dict = {}

# for document in data['Judul']:
#     for term in document:
#         if term not in term_dict:
#             term_dict[term] = ' '

# for document in data['Isi Berita']:
#     for term in document:
#         if term not in term_dict:
#             term_dict[term] = ' '
            
# print(len(term_dict))
# print("------------------------")

# for term in term_dict:
#     term_dict[term] = stemmed_wrapper(term)
#     print(term,":" ,term_dict[term])
    
# print(term_dict)
# print("------------------------")


# # apply stemmed term to dataframe
# def get_stemmed_term(document):
#     return [term_dict[term] for term in document]

Streaming output truncated to the last 5000 lines.
memukau : pukau
berlenggang : lenggang
panggung : panggung
baswedanmenjelaskan : baswedanmenjelaskan
jugaadzan : jugaadzan
syahdujis : syahdujis
bakauheni : bakauheni
covidmenteri : covidmenteri
ira : ira
puspadewi : puspadewi
bakaheuni : bakaheuni
terkaitapapun : terkaitapapun
konsekuensinya : konsekuensi
diperintahkan : perintah
jugawagub : jugawagub
masiftentu : masiftentu
ltmpt : ltmpt
jalurutbk : jalurutbk
sbmptn : sbmptn
komputer : komputer
jugapersyaratan : jugapersyaratan
snmptn : snmptn
sajapelaksanaan : sajapelaksanaan
berlangsungkarena : berlangsungkarena
utbk : utbk
mengkikuti : mengkikuti
p : p
widyobroto : widyobroto
jugainformasi : jugainformasi
terjadinyapeningkatan : terjadinyapeningkatan
pengingat : ingat
risikocovidmasih : risikocovidmasih
adapemerintah : adapemerintah
artisfitri : artisfitri
tropicaunggah : tropicaunggah
melihatkan : lihat
janin : janin
jelasini : jelasini
adek : adek
kakasada : kakasada
getergeteri

In [ ]:
data['Judul'] = data['Judul'].swifter.apply(get_stemmed_term)
data['Isi Berita'] = data['Isi Berita'].swifter.apply(get_stemmed_term)
data.head()

In [ ]:
data["Judul"]= data["Judul"].str.join(" ")
data["Isi Berita"]= data["Isi Berita"].str.join(" ")
data.head()

,Judul,Isi Berita,Label
0,kapolri vaksinasi booster lansia maksimal,cianjur kapolri jenderal listyo sigit prabowo ...,0
1,vaksin sintas covid papar omicron,jakarta gejala infeksi covid variant omicron g...,1
2,infeksi covid bahaya komorbid,dokter spesialis sakit konsultan sakit tropik ...,0
3,mnc peduli lotte mart gelar sentra vaksin boos...,jakarta lotte mart mnc peduli helat sentra vak...,0
4,kapolda metro tinjau vaksinasi lansia alunalun...,bekas kapolda metro jaya irjen fadil imran dam...,0


In [ ]:
data.to_csv("Data Clean.csv")